In [1]:
import jellyfish as jf
import pandas as pd
import numpy as np
import pickle
from copy import deepcopy
import building_tokenizer as toker
from collections import Counter, defaultdict, OrderedDict
from sklearn import preprocessing
from sklearn.decomposition import PCA
from ontolcs import ontolcs
from operator import itemgetter
import csv
import re
import operator
import math
from pyjarowinkler import distance
from stringscore import liquidmetal
from itertools import chain
import os
import get_schema2
reload(get_schema2)
#from get_schema2 import tagList, tagsetList
from brick_parser import tagList, tagsetList, equalDict
from datetime import datetime

from matplotlib import pyplot as plt

f = lambda x:x.replace('_', ' ')
tagsetList = map(f, tagsetList)
termList = list(set(tagList+tagsetList))

from simple_map_reduce import SimpleMapReduce
import multiprocessing

import pprint
pp = pprint.PrettyPrinter()

Init Graph
Load Brick.ttl
Load BrickFrame.ttl


In [11]:
buildingName = 'ghc'
ucsdBuildings = ['ebu3b', 'ap_m', 'bsb', 'bonner']

if buildingName in ucsdBuildings:

    naeDict = dict()
    naeDict['bonner'] = ["607", "608", "609", "557", "610"]
    naeDict['ap_m'] = ['514', '513','604']
    naeDict['bsb'] = ['519', '568', '567', '566', '564', '565']
    naeDict['ebu3b'] = ["505", "506"]
    naeList = naeDict[buildingName]

    labeledFile = 'metadata/' + buildingName + '_sensor_types_location.csv'
    with open(labeledFile, 'rb') as fp:
        #truthDF = pd.read_excel(fp)
        truthDF = pd.DataFrame.from_csv(fp)
        #truthDF = truthDF.set_index(keys='Unique Identifier')

    wordFeatFile = 'data/wordfeat_'+buildingName+'.pkl'

    tokenType = 'Alphanumeric'
    #tokenType = 'NumAsSingleWord'
    #tokenType = 'NoNumber'
    tokenTypeList = ['NoNumber', 'Alphanumeric', 'AlphaAndNum', 'NumAsSingleWord']

    bacnetTypeMapDF = pd.DataFrame.from_csv('metadata/bacnettype_mapping.csv')
    unitMap = pd.read_csv('metadata/unit_mapping.csv').set_index('unit')
    for val in Counter(unitMap.keys()).values():
        if val>1:
            "Unit map file ERROR"
            assert(False)

    tokenType = 'NoNumber'
    trueDF = pd.DataFrame.from_csv('metadata/'+buildingName+'_sensor_types_location.csv')
    sensorDF, nameList, jcinameList, descList, unitList, _, wordList = \
    toker.structure_metadata(buildingName=buildingName, tokenType=tokenType, \
                             validSrcidList=trueDF.index.tolist(), withDotFlag=False)
else:
    filename = 'metadata/'+buildingName+'_sensor_types_location.csv'
    #   filename = 'metadata/%s_sensor_types_location.csv'%buildingName
    df = pd.read_csv(filename)
    sentenceList = list()
    
    tokenType = 'NoNumber'
    for raw in df['bas_raw'].tolist():
        sentenceList.append(toker.tokenize(tokenType, raw))
    adder = lambda x,y:x+y
    wordList = list(set(reduce(adder,sentenceList,[])))

In [13]:
# Ported from Quicksilver's scoreForAbbreviation algorithm
# https://github.com/quicksilver/Quicksilver/

SCORE_NO_MATCH = 0.0
SCORE_MATCH = 1.0
SCORE_TRAILING = 0.9
SCORE_BUFFER = 0.15
WHITESPACE_CHARACTERS = ' \t'


def quicksilver_score2(string, abbrev):
    abbrev = abbrev.lower()
    abbrev_len = len(abbrev)
    string_len = len(string)

    # deduct some points for all remaining letters
    if abbrev_len == 0:
        return SCORE_TRAILING
    if abbrev_len > string_len:
        return SCORE_NO_MATCH

    # Search for steadily smaller portions of the abbreviation
    for i in range(abbrev_len, 0, -1):
        try:
            index = string.lower().index(abbrev[:i])
        except ValueError:
            continue  # Not found

        if index + abbrev_len > string_len:
            continue

        next_string = string[index + i:]
        next_abbrev = abbrev[i:]

        # Search what is left of the string with the rest of the abbreviation
        remaining_score = quicksilver_score2(next_string, next_abbrev)

        if remaining_score > 0:
            result_score = index + i

            # ignore skipped characters if is first letter of a word
            if index > 0:  # if some letters were skipped
                if string[index - 1] in WHITESPACE_CHARACTERS:
                    for j in range(index - 1):
                        c = string[j]
                        result_score -= SCORE_MATCH \
                                        if c in WHITESPACE_CHARACTERS \
                                        else SCORE_BUFFER
                elif 'A' <= string[index] <= 'Z':
                    for j in range(index):
                        c = string[j]
                        result_score -= SCORE_MATCH \
                                        if 'A' <= c <= 'Z' \
                                        else SCORE_BUFFER
                else:
                    result_score -= index

            result_score += remaining_score * len(next_string)
            result_score /= string_len
            return result_score

    return SCORE_NO_MATCH

In [14]:
#Quicksilver: String to abbreviation ranking algorithm
def quicksilver_score(s, a, offset=0):
    s = s.lower()
    a = a.lower()
    if len(a)==0:
        return 0.9
    if len(a)>len(s):
        return 0
    
    for i in reversed(range(1,len(a)+1)):
        subA = a[0:i]
        if subA in s:
            index = s.index(subA)
        else:
            continue
        if index<0:
            continue
        if index+len(a) > len(s) + offset:
            continue
            
        nextS = s[index+len(subA):]
        nextA = None
        if i>=len(a):
            nextA = ''
        else:
            nextA = a[i:]
        remainingScore = quicksilver_score(nextS, nextA, offset+index)
        if remainingScore>0:
            score = len(s) - len(nextS)
            if index is not 0:
                #c = ord(s[index-1:][0])
                #if c==32 or c==9:
                if s[index-1]==' ':
                    for j in reversed(range(0,index-2)):
                        #c = ord(s[j])
                        #score -= (1 if c==32 or c==9 else 0.15)
                        score -= (1 if s[j]==' ' else 0.15)
                else:
                    score -= index
            score += remainingScore * len(nextS)
            score /= len(s)
            return score
    return 0

In [15]:
#Quicksilver: String to abbreviation ranking algorithm
def modified_quicksilver_score(s, a, offset=0, aPos=0):
    s = s.lower()
    a = a.lower()
    if len(a)==0:
        return 0.9 - 0.01*sum([1 if ' '==oneS else 0 for oneS in s])
    if len(a)>len(s):
        return 0
    
    for i in reversed(range(1,len(a)+1)):
        subA = a[0:i]
        if subA in s:
            index = s.index(subA)
        else:
            continue
        if index<0:
            continue
        if index+len(a) > len(s) + offset:
            continue
            
        nextS = s[index+len(subA):]
        nextA = None
        if i>=len(a):
            nextA = ''
        else:
            nextA = a[i:]
        remainingScore = modified_quicksilver_score(nextS, nextA, offset+index, aPos+i)
        
        
        if remainingScore>0:
            currS = s[:index+len(subA)]
            if ' ' in currS:
                spaceIndices = [i for i,c in enumerate(currS) if c==' ']
                if len(spaceIndices)>=1 and aPos==0:# and False:
                    score = len(s[spaceIndices[-1]:]) - len(nextS) - 0.01*(len(spaceIndices)-1)
                else:
                    score = len(s) - len(nextS)
            else:
                score = len(s) - len(nextS)
            
            #score = len(s) - len(nextS)
            
            #print s, '/', a, '/', currS, '/', aPos
            #print score
            
            if index is not 0:
                #c = ord(s[index-1:][0])
                #if c==32 or c==9:
                if s[index-1]==' ':
                    for j in reversed(range(0,index-2)):
                        #c = ord(s[j])
                        #score -= (1 if c==32 or c==9 else 0.15)
                        if s[j]==' ':
                            score -= 1  
                        else:
                            score -= 0.15
                else:
                    score -= index
            score += remainingScore * len(nextS)
            score /= len(s)
            return score
    return 0

#print modified_quicksilver_score('differential pressure', 'dp')
#print modified_quicksilver_score('dewpoint setpoint', 'dp')
print modified_quicksilver_score('chilled water pump command', 'chwp')
print '-------'
print modified_quicksilver_score('chilled water pump', 'chwp')
print '-------'
print modified_quicksilver_score('warmest zone temperature sensor', 'zn')
print '-------'
print modified_quicksilver_score('zone temperature sensor', 'zn')
print '-------'

0.913076923077
-------
0.925
-------
0.635483870968
-------
0.852173913043
-------


In [16]:
#Longest common substring
def lcs_len(X, Y):
    m = len(X)
    n = len(Y)
    # An (m+1) times (n+1) matrix
    C = [[0] * (n + 1) for _ in range(m + 1)]
    for i in range(1, m+1):
        for j in range(1, n+1):
            if X[i-1] == Y[j-1]: 
                C[i][j] = C[i-1][j-1] + 1
            else:
                C[i][j] = max(C[i][j-1], C[i-1][j])
    lenList = [subC[-1] for subC in C]
    return max(lenList)

In [19]:
if buildingName in ucsdBuildings:
    unitMap = pd.read_csv('metadata/unit_mapping.csv').set_index('unit')

    tokenType = 'NoNumber'
    sensorDF, nameList, jcinameList, descList, unitList, _, wordList = \
    toker.structure_metadata(buildingName=buildingName, tokenType=tokenType, 
                             validSrcidList=trueDF.index.tolist(), withDotFlag=False)

    #wordList: List of all words after tokenizing metadata of a building
    #alphaWordList: List of alphabetical words only
    alphaWordList = list()
    for word in wordList:
        alphaWordList += re.findall('[a-zA-Z]+', word)
    addFunc = lambda a,b:a+b
    wordList = list(set(alphaWordList+reduce(addFunc,[unit.split() for unit in unitMap['word'].tolist() if type(unit)==str])))
    try:
        wordList.remove(np.nan)
    except:
        pass

    print len(wordList)

In [20]:
def apply_metric_schema(metric, wordList, termList):
    calcDistPerWord = lambda s:dict([(term,apply(metric,[s,term])) for term in termList])
    constructTuple = lambda word:tuple((word,apply(calcDistPerWord,[word])))
    distDictDict = dict(map(constructTuple,wordList))
    return distDictDict

In [21]:
def map_func(metric, word, termList):
    calcDistPerWord = lambda s:dict([(term,apply(metric,[s,term])) for term in termList])
    #constructTuple = lambda word:tuple((word,apply(calcDistPerWord,[word])))
    return tuple((word, apply(calcDistPerWord,[word])))

In [22]:
metricDict = OrderedDict()
metricDict['ontolcs'] = ontolcs
#jaroLambda = lambda a,b: distance.get_jaro_distance(unicode(a), unicode(b), scaling=0.1)
jaroLambda = lambda a,b: jf.jaro_winkler(unicode(a), unicode(b))
metricDict['jaro'] = jaroLambda
#quicksilverLambda = lambda a,b: liquidmetal.score(unicode(b), unicode(a))
quicksilverLambda = lambda a,b: modified_quicksilver_score(unicode(b), unicode(a))
metricDict['quicksilver'] = quicksilverLambda
#matchRatingLambda = lambda a,b: distance.get_jaro_distance(jf.match_rating_codex(unicode(b)), unicode(a))
#metricDict['match_rating'] = matchRatingLambda

#lcsLambda = lambda a,b:lcs_len(a,b)/np.sqrt(float(len(a))*len(b))
#metricDict['lcs'] = lcsLambda

editLambda = lambda a,b:1-jf.levenshtein_distance(unicode(a), unicode(b))/float(max(len(a),len(b)))
metricDict['edit'] = editLambda

In [23]:
totalTagsetDistDict = dict()
totalTagTagsetDistDict = dict()
totalTagDistDict = dict()

mpFlag = False
if mpFlag:
    print datetime.now()
    totalTagsetDistDict = dict()
    totalTagDistDict = dict()
    workerNum = 4
    pool = multiprocessing.Pool(workerNum)
    for key, metric in metricDict.items():
        #localMapFunc = lambda x:map_func(deepcopy(metric),x, deepcopy(tagsetList))
        print key, datetime.now()

        #Tagset Dist
        def localTagsetMapFunc(x):
            return map_func(metric,x, tagsetList)
        mapResult = pool.map(localTagsetMapFunc, wordList, chunksize=len(wordList)/workerNum+1)
        totalTagsetDistDict[metric] = dict(mapResult)
    print "Finished: ", datetime.now()
else:
    totalDistDict = dict()
    for key, metric in metricDict.items():
        #totalTagsetDistDict[key] = apply_metric_schema(metric, wordList, tagsetList)
        #totalTagDistDict[key] = apply_metric_schema(metric, wordList, tagList)
        totalTagTagsetDistDict[key] = apply_metric_schema(metric, wordList, list(set(tagList+tagsetList)))

In [24]:
def normalize_dist_dict(totalDistDict):
    #totalDistDict = deepcopy(totalTagsetDistDict)
    for metric, tagsetDistDict in totalDistDict.items():
        newTagsetDistDict = dict()
        for i, (word, distDict) in enumerate(tagsetDistDict.items()):
            maxVal = float(max(distDict.values()))
            minVal = float(min(distDict.values()))
            distDict = OrderedDict(distDict)
            if maxVal==0:
                newTagsetDistDict[word] = dict((key,0.5) for key in distDict.keys())
            else:
                normalize = lambda datum: (datum-minVal)/maxVal
                values = map(normalize,distDict.values())
                newTagsetDistDict[word] = dict((key,val) for key, val in \
                                               zip(distDict.keys(), values))
        totalDistDict[metric] = newTagsetDistDict
    return totalDistDict

#totalTagsetDistDict = normalize_dist_dict(totalTagsetDistDict)
#totalTagDistDict = normalize_dist_dict(totalTagDistDict)
totalTagTagsetDistDict = normalize_dist_dict(totalTagTagsetDistDict)

In [25]:
tagDF = pd.DataFrame(index=wordList)
tagsetDF = pd.DataFrame(index=wordList)
def distdict_df(totalDistDict, df):
    for metricName in totalDistDict.keys():
        valList = list()
        for word in df.index.tolist():
            maxVal = max(totalDistDict[metricName][word].values())
            maxIdx = totalDistDict[metricName][word].values().index(maxVal)
            maxKey = totalDistDict[metricName][word].keys()[maxIdx]
            valList.append(maxKey)
        df[metricName] = valList
    return df

#tagsetDF = distdict_df(totalTagsetDistDict, tagsetDF)
#tagDF = distdict_df(totalTagDistDict, tagDF)
tagtagsetDF = distdict_df(totalTagTagsetDistDict, tagDF)

#tagsetDF.to_excel('result/tagset_comp_result.xlsx')
#tagDF.to_excel('result/tag_comp_result.xlsx')
tagtagsetDF.to_excel('result/tagtagset_comp_result.xlsx')

In [26]:
totalFeatDict = dict()
metricNum = len(metricDict.keys())
zeroFeat = [0 for i in range(metricNum)]
for word in wordList:
    featDict = OrderedDict()
    for term in termList:
        featDict[term] = deepcopy(zeroFeat)
    totalFeatDict[word] = featDict
for metricIdx, (metricName, totalDistDict) in enumerate(totalTagTagsetDistDict.items()):
    for word, distDict in totalDistDict.items():
        for term, dist in distDict.items():
            totalFeatDict[word][term][metricIdx] = dist

In [27]:
scoreDictDict = dict()

notUseSynonymList = equalDict.keys()
useSynonymList = ['vav','ahu', 'vfd', 'crac']
pickithElements = lambda xList,i:[x[i] for x in xList]

scoreSummaryDict = dict()

oneDimExtract = lambda x:x[i]

for word in wordList:
    #word = 'chwdp'
    pcaN = 2
    pca = PCA(n_components=pcaN)
    pca.fit(totalFeatDict[word].values())
    for j, component in enumerate(pca.components_):
        if np.inner(component, np.ones(len(metricDict)))<0:
            pca.components_[j] = -component
    values = pca.transform(totalFeatDict[word].values())
    '''
    scoreDict = dict((k,v) for k,v in zip(totalFeatDict[word].keys(), map(sum,values)))
    orderedScoreDict = OrderedDict(sorted(scoreDict.items(), key=itemgetter(1), reverse=True)[0:10])
    '''
    '''
    subOrderedScoreDictList = list()
    for i in range(0, pcaN):
        oneDimExtract = lambda x:x[i]
        scoreDict = dict((k,v) for k,v in zip(totalFeatDict[word].keys(), map(oneDimExtract,values)))
        subOrderedScoreDictList.append(OrderedDict(sorted(scoreDict.items(), key=itemgetter(1), reverse=True)[0:5]))
    
    orderedScoreDict = dict()
    for subOrderedScoreDict in subOrderedScoreDictList:
        orderedScoreDict = dict(orderedScoreDict.items()+subOrderedScoreDict.items())
    '''
    orderedScoreDict = defaultdict(float)
    featDict = OrderedDict(totalFeatDict[word])
    for i in range(0,len(featDict.values()[0])):
        valueList = pickithElements(featDict.values(),i)
        pickedValueList = sorted(valueList, reverse=True)[0:3]
        addedIndices = list()
        for pickedValue in pickedValueList:
            pickedIndices = [idx for idx, val in enumerate(valueList) if val==pickedValue]
            for pickedIdx in pickedIndices[0:3]:
                if pickedIdx not in addedIndices:
                    addedIndices.append(pickedIdx)
                    orderedScoreDict[featDict.keys()[pickedIdx]] += pickedValue
    orderedScoreDict = dict(orderedScoreDict)
    
    scoreDictDict[word] = orderedScoreDict
    scoreSummaryDict[(word,orderedScoreDict.keys()[0])] = orderedScoreDict.values()[0]

In [28]:
for metricName, distDict in totalTagTagsetDistDict.items():
    print '===', metricName
    distDict = distDict['dp']
    values = sorted(distDict.values(), reverse=True)[0:5]
    printedIndices = list()
    for val in values:
        indices = [idx for idx,value in enumerate(distDict.values()) if value==val]
        print val, len(indices)
        for idx in indices[0:5]:
            if not (idx in printedIndices):
                print distDict.keys()[idx]
                printedIndices.append(idx)

=== edit
1.0 2
di
dc
1.0 2
0.666666666667 2
damper
dew
0.666666666667 2
0.5 9
code
mode
duct
dewpoint
open
=== ontolcs
1.0 2
di
dc
1.0 2
0.960698689956 1
pv
0.876232064878 1
dew
0.829070492826 4
led
pre
pir
vfd
=== jaro
1.0 1
damper
0.964285714286 1
dewpoint
0.918367346939 1
damper command
0.914285714286 2
dewpoint sensor
damper position
0.914285714286 2
=== quicksilver
1.77150924593 1
dew point setpoint
1.75289083633 1
differential pressure
1.74871474445 1
differential pressure sensor
1.74861034215 1
differential pressure setpoint
1.74182419286 1
differential pressure integral time


In [29]:
print quicksilverLambda('dp', 'differential pressure proportional band setpoint')
print quicksilverLambda('dp', 'differential pressure')

print quicksilver_score2('differential pressure proportional band setpoint', 'dp')
print quicksilver_score2('differential pressure', 'dp')

0.876666666667
0.895238095238
0.894791666667
0.888095238095


In [30]:
with open('data/'+buildingName+'_str_score_dict.pkl', 'wb') as fp:
    pickle.dump(scoreDictDict, fp)
    
tuplelist2list = lambda x:list(x)
addFunc = lambda x,y:x+y
with open('result/'+buildingName+'_str_comp_score.csv', 'wb') as fp:
    writer = csv.writer(fp)
    writer.writerow(['word in metadata', '[tagset, score]'])
    for key, scoreDict in scoreDictDict.items():
        writer.writerow([key]+reduce(addFunc,map(list,scoreDict.items())))

In [31]:
with open('result/'+buildingName+'_wordmapping.csv', 'wb') as fp:
    writer = csv.writer(fp)
    for key, score in scoreSummaryDict.items():
        writer.writerow([key[0], key[1],score])

In [32]:
vectorList = list()
pairList = list()
for word, featDict in totalFeatDict.items():
    for tagTagset, vector in featDict.items():
        vectorList.append(vector)
        pairList.append((word,tagTagset))
learningSampleSet = [
('alm', 'alarm'),
('deceleration', 'deceleration'),
('stndby', 'standby'),
('freq', 'frequency'),
('chwdp', 'chilled water differential pressure sensor'),
]
learningVectorList = list()
learningLabelList = list()
origIndexList = list()
learningPairList = list()
for word, tagTagset in learningSampleSet:
    print word, tagTagset
    for i, pair in enumerate(pairList):
        if pair[0]==word:
            learningVectorList.append(vectorList[i])
            origIndexList.append(i)
            if pair[1]==tagTagset:
                learningLabelList.append(1)
            else:
                learningLabelList.append(0)

alm alarm
deceleration deceleration
stndby standby
freq frequency
chwdp chilled water differential pressure sensor


In [33]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.linear_model import LogisticRegression
learner = AdaBoostClassifier()
#learner = RandomForestClassifier()
#learner = DecisionTreeClassifier()
#learner = BernoulliNB()
#learner = LogisticRegression()
learner.fit(learningVectorList, learningLabelList)

predLabelList = learner.predict(vectorList)
predScoreList = learner.predict_proba(vectorList)

/home/jbkoh/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


ValueError: Found array with 0 feature(s) (shape=(1, 0)) while a minimum of 1 is required.

In [ ]:
with open('result/temporary_word_classification_result.csv', 'wb') as fp:
    writer = csv.writer(fp)
    for i, (pair, label) in enumerate(zip(pairList,predLabelList)):
        writer.writerow([pair[0], pair[1], label])

with open('result/temporary_word_classification_score_result.csv', 'wb') as fp:
    writer = csv.writer(fp)
    for i, (pair, score) in enumerate(zip(pairList,predScoreList)):
        writer.writerow([pair[0], pair[1], "%.3f" % score[1]])
        
with open('result/temporary_word_classification_result_label1.csv', 'wb') as fp:
    writer = csv.writer(fp)
    for i, (pair, label) in enumerate(zip(pairList,predLabelList)):
        if label==1:
            writer.writerow([pair[0], pair[1], label])

foundList = list()
with open('result/temporary_word_classification_result_label1_nonredundant.csv', 'wb') as fp:
    writer = csv.writer(fp)
    
    for i, (pair, label) in enumerate(zip(pairList,predLabelList)):
        if pair[0] in foundList:
            continue
        if label==1:
            writer.writerow([pair[0], pair[1], label])
            foundList.append(pair[0])
notfoundList = list()
with open('result/temporary_word_classification_result_label1_notfound.csv', 'wb') as fp:
    writer = csv.writer(fp)
    for i, (pair, label) in enumerate(zip(pairList,predLabelList)):
        if pair[0] not in foundList:
            writer.writerow([pair[0], pair[1], predScoreList[i][1]])
            if pair[0] not in notfoundList:
                notfoundList.append(pair[0])
print "Found: ", len(foundList)
print "Not Found: ", len(notfoundList)